In [1]:
# Generic imports
import pandas as pd
import plotly
import time
import plotly.graph_objects as go

# 5G visualization logic
import trace_visualizer
import trace_plotting

In [2]:
traces_to_compare = [
    'doc/UP_example_analysis.pbz2',
    'doc/UP_example_analysis_2.pbz2',
]

trace_names = [
    'Data trace 1',
    'Data trace 2',
]

output_trace = 'doc/latency_comparison.html'

In [3]:
summary = []
histogram_data = []
for idx,trace in enumerate(traces_to_compare):
    print('Loading {0}'.format(trace))
    data = trace_plotting.decompress_pickle(trace)
    packets = data['packets']
    n3_packets = data['packets_n3']
    n6_packets = data['packets_n6']
    (hist_array, hist_bins) = data['histogram']
    bin_size = data['histogram_bin_size']
    
    summary.append((packets.delay_us.mean(), packets.delay_us.min(), packets.delay_us.max(), len(n3_packets), len(n6_packets)))
    histogram_data.append((hist_array, hist_bins))

Loading doc/UP_example_analysis.pbz2
Loading doc/UP_example_analysis_2.pbz2


In [4]:
df = pd.DataFrame(summary, columns =['Mean delay', 'Min delay', 'Max delay', 'N3 packets', 'N6 packets'])
df['lost'] = (df['N6 packets'] - df['N3 packets']).abs()
df['Trace'] = trace_names
display(df)

,Mean delay,Min delay,Max delay,N3 packets,N6 packets,lost,Trace
0,3814.851181,3602.981567,4461.050034,92,91,1,Data trace 1
1,3798.314503,3761.053085,3937.005997,10,10,0,Data trace 2


In [5]:
fig = go.Figure()
for idx,histogram_data_item in enumerate(histogram_data):
    hist_array = histogram_data_item[0]
    hist_bins = histogram_data_item[1]
    histogram_line = go.Scatter(
        x=hist_bins,
        y=hist_array,
        mode='lines',
        name=trace_names[idx],
        line_shape='spline',
        opacity=0.75,
        showlegend=True)
    fig.add_trace(histogram_line)
    
fig.update_xaxes(title_text='Delay (μs)')
fig.show()
fig.write_html(output_trace)